In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import streamlit as st
import psutil

from mainnet_launch.constants import *

from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
from mainnet_launch.database.schema.views import *


# for autopool in ALL_AUTOPOOLS:
#     df = fetch_autopool_destination_state_df(autopool)
#     print(f"{autopool.name} has {len(df)} destination states {df['datetime'].max()}")

In [ ]:
# this tells me that the destination states table is curret

In [14]:
chain = ETH_CHAIN
autopool_and_destinations_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            AutopoolDestinations,
            [
                AutopoolDestinations.destination_vault_address,
                AutopoolDestinations.autopool_vault_address,
            ],
        ),
        TableSelector(
            Destinations,
            Destinations.underlying_name,
            join_on=AutopoolDestinations.destination_vault_address == Destinations.destination_vault_address,
        ),
    ],
    where_clause=(Destinations.chain_id == chain.chain_id)
    & (AutopoolDestinations.autopool_vault_address.in_([a.autopool_eth_addr for a in ALL_AUTOPOOLS_DATA_ON_CHAIN])),
)

autopool_and_destinations_df

,destination_vault_address,autopool_vault_address,underlying_name
0,0xe3AE2Ab9AE8ADe1B4940dd893C9339401bEe61A1,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,Gyroscope ECLP wstETH/cbETH
1,0x90300b02b162F902B9629963830BcCCdeEd71113,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,Balancer weETH/ezETH/rswETH
2,0x60339056EC88996e41757E05a798310E46972cca,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,Balancer rsETH / ETHx
3,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH
4,0x6DcB6797b1C0442587c2ad79745ef7BB487Fc2E2,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,Gyroscope ECLP wstETH/wETH
...,...,...,...
109,0xe3AE2Ab9AE8ADe1B4940dd893C9339401bEe61A1,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,Gyroscope ECLP wstETH/cbETH
110,0x4142e3a17391676C66DDf1285E43889F168EE237,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,Balancer osETH-waWETH
111,0x40219bBda953ca811d2D0168Dc806a96b84791d9,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,Balancer weETH/rETH StablePool
112,0xFDa49984eb4eA4075b8b451032849347C633E94b,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,Balancer osETH/wETH StablePool


In [15]:
several_destination_token_values_df = get_full_table_as_df(
    DestinationTokenValues,
)
# where_clause=DestinationTokenValues.destination_vault_address.in_(
#     autopool_and_destinations_df["destination_vault_address"].unique()
# ),


several_destination_token_values_df

,block,chain_id,token_address,destination_vault_address,denominated_in,spot_price,quantity
0,22196950,1,0xdAC17F958D2ee523a2206206994597C13D831ec7,0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0.999781,7.995051e+05
1,22859279,1,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,0xa345ceECCF8fe6aE33fe1D655B4806492251c2A8,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0.999883,1.161878e+06
2,22644092,1,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.000176,1.932898e+08
3,22308953,1,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,0xa345ceECCF8fe6aE33fe1D655B4806492251c2A8,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.000080,7.209925e+05
4,22321187,1,0x83F20F44975D03b1b09e64809B757c47f942BEeA,0xAd7b8dB1efEe849c67060c6b10BC7D6D3C607aa5,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.157446,6.964177e+06
...,...,...,...,...,...,...,...
82861,43163463,146,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,0x3D35b1E0Fe434848490427CE4DE13b6839365Bd2,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,1.000261,4.040410e+05
82862,43163463,146,0xd3DCe716f3eF535C5Ff8d041c1A41C3bd89b97aE,0x2Af19422D9f81A4c71306a409A540fC63d7f623B,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,0.998037,1.232306e+05
82863,43163463,146,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,0x310CCB9eC996A228cd6F4Eb2E70769ccDbeD163D,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,0.999939,6.695179e+05
82864,43163463,146,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,0xD4E59aa6dfFE9Acd541f8B0a827683a6656f62a1,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,1.000261,2.311364e+05


In [20]:
several_destination_token_values_df.groupby("destination_vault_address")["block"].nunique().sort_values(
    ascending=False
).head(10)

destination_vault_address
0xa7569A44f348d3D70d8ad5889e50F78E33d80D35    1241
0xbB2d2dd491204a86ec10a1a6972F940B34fE060e    1235
0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee    1235
0x8acA8accFb69adeff607431E0F25466B7b76a8Ad    1235
0xc73aF8064EcbfcCBF687cF36a8ABaFf011eAB1fC    1235
0xC099899d0278CE83976218Cbe58D01dD382dcA32    1235
0xCADe3658eCEA14e4da22ef81BE335E9783E68848    1235
0xb949C40F18fB52747d96850696c954718168fde3    1235
0x7583b1589aDD33320366A48A92794D77763FAE9e    1235
0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c    1235
Name: block, dtype: int64

In [17]:
several_destination_token_values_df[
    several_destination_token_values_df["destination_vault_address"] == "0x90300b02b162F902B9629963830BcCCdeEd71113"
]

,block,chain_id,token_address,destination_vault_address,denominated_in,spot_price,quantity


In [ ]:
pass

In [ ]:
several_destination_states_df.groupby("destination_vault_address")["block"].count()

In [ ]:
break

In [ ]:
from mainnet_launch.database.schema.postgres_operations import _exec_sql_and_cache

In [ ]:
query = """SELECT DISTINCT ON (DATE_TRUNC('day', datetime))
    DATE_TRUNC('day', datetime) AS day,
    block AS max_block,
    datetime AS datetime_of_max
FROM blocks
WHERE chain_id = :chain_id
  AND block BETWEEN :start_block AND :end_block
ORDER BY DATE_TRUNC('day', datetime), block DESC;"""

query = """SELECT DISTINCT ON (DATE_TRUNC('day', datetime))
    DATE_TRUNC('day', datetime) AS day,
    block AS max_block,
    datetime AS datetime_of_max
FROM blocks
WHERE chain_id = 1
  AND block BETWEEN 20722908 AND 100000000
ORDER BY DATE_TRUNC('day', datetime), block DESC;"""

from mainnet_launch.database.schema.postgres_operations import _exec_sql_and_cache

block_df = _exec_sql_and_cache(query)

In [ ]:
block_df

In [ ]:
destination_state_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationStates,
        ),
        TableSelector(
            Destinations,
            select_fields=[
                Destinations.underlying_name,
                Destinations.exchange_name,
                Destinations.pool,
            ],
            join_on=(Destinations.destination_vault_address == DestinationStates.destination_vault_address)
            & (Destinations.chain_id == DestinationStates.chain_id),
        ),
        TableSelector(
            AutopoolDestinations,
            select_fields=[
                AutopoolDestinations.autopool_vault_address,
            ],
            join_on=(AutopoolDestinations.destination_vault_address == DestinationStates.destination_vault_address)
            & (AutopoolDestinations.chain_id == DestinationStates.chain_id),
        ),
        TableSelector(
            Blocks,
            [Blocks.datetime],
            (DestinationStates.block == Blocks.block) & (DestinationStates.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(AutopoolDestinations.autopool_vault_address == AUTO_USD.autopool_eth_addr),
    order_by=Blocks.datetime,
)

destination_state_df

In [ ]:
a_dest = "0x7583b1589aDD33320366A48A92794D77763FAE9e"

one_autoUSD_dest = get_full_table_as_df(
    DestinationStates, where_clause=DestinationStates.destination_vault_address == a_dest
)
one_autoUSD_dest

In [ ]:
auto_DOLA_dest = "0x2fa4C8a4aCda7746778dbd2f3B2Fc5ca41D9b171"

auto_dola_dest_df = get_full_table_as_df(
    DestinationStates, where_clause=DestinationStates.destination_vault_address == auto_DOLA_dest
)
auto_dola_dest_df

In [ ]:
auto_dola_dest_df["block"].max()

In [ ]:
# auto DOLA is current

In [ ]:
# autoUSD destination a_dest does not have any entires after lbock 22865242 july 7ths

In [ ]:
one_autoUSD_dest["block"].max()

In [ ]:
destination_state_df["datetime"].max()

# destination_state_df = merge_tables_as_